# DBATools Log Shipping Demo
## Configure Log Shipping

In [29]:
Remove-DbaDbLogShipping -PrimarySqlInstance 'localhost' -SecondarySqlInstance 'localhost\tony' -Database LSTest;
Remove-DbaDatabase -SqlInstance 'localhost\tony' -Database LSTestDR -Confirm:$false;



ComputerName : FD-PF33VX6J
InstanceName : TONY
SqlInstance  : FD-PF33VX6J\TONY
Database     : LSTestDR
Status       : Dropped





In [30]:
Remove-Item -Path "C:\LogShipping\LSTest\*";
Remove-Item -Path "\\localhost\LogShippingDest\LSTest\*";
Remove-Item -Path "C:\LogShippingDest\LSTest\*";

$params = @{
 SourceSqlInstance = 'localhost'
 DestinationSqlInstance = 'localhost\tony'
 Database = 'LSTest'
 SharedPath= '\\localhost\LogShipping'
 LocalPath= 'C:\logshipping'
 BackupScheduleFrequencyType = 'daily'
 BackupScheduleFrequencyInterval = 1
 BackupScheduleDisabled = $true
 CompressBackup = $true
 CopyScheduleFrequencyType = 'daily'
 CopyScheduleFrequencyInterval = 1
 CopyScheduleDisabled = $true
 GenerateFullBackup = $true
 RestoreScheduleFrequencyType = 'daily'
 RestoreScheduleFrequencyInterval = 1
 RestoreScheduleDisable = $true
 SecondaryDatabaseSuffix = 'DR'
 CopyDestinationFolder = '\\localhost\LogShippingDest'
 Force = $true
 Standby = $true
 DisconnectUsers = $true
 }

Invoke-DbaDbLogShipping @params;




PrimaryInstance   : FD-PF33VX6J
SecondaryInstance : FD-PF33VX6J\TONY
PrimaryDatabase   : LSTest
SecondaryDatabase : LSTestDR
Result            : Success
Comment           : 





## Select from Primary and Secondary Tables

In [31]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;



Column1          Col1 Col2 Col3    
-------          ---- ---- ----    
FD-PF33VX6J         1 Log  Shipping
FD-PF33VX6J\TONY    1 Log  Shipping




## Insert Into Primary Table

In [32]:
$insert = @"
INSERT INTO TestTable
(Col2
,Col3)
VALUES
('Is'
,'Resilient')
"@
$primary = "localhost";
$primarydb = "LSTest";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $insert;

## Select from Primary and Secondary Tables

In [33]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;


Column1          Col1 Col2 Col3     
-------          ---- ---- ----     
FD-PF33VX6J         1 Log  Shipping 
FD-PF33VX6J         2 Is   Resilient
FD-PF33VX6J\TONY    1 Log  Shipping 




## Execute Log Shipping Jobs

In [34]:
Start-DbaAgentJob -SqlInstance "localhost" -Job LSBackup_LSTest;
Start-Sleep -Seconds 5;
Start-DbaAgentJob -SqlInstance "localhost\tony" -Job LSCopy_localhost_LSTest;
Start-Sleep -Seconds 5;
Start-DbaAgentJob -SqlInstance "localhost\tony" -Job LSRestore_localhost_LSTest;


Name                      Owner                Category                  Enabled    CurrentRunStatus     DateCreated   
----                      -----                --------                  -------    ----------------     -----------   
LSBackup_LSTest           sa                   Log Shipping              False      Idle                 3/15/2022 3...


LSCopy_localhost_LSTest   sa                   Log Shipping              False      Executing            3/15/2022 3...


LSRestore_localhost_LS... sa                   Log Shipping              False      Executing            3/15/2022 3...




## Select from Primary and Secondary Tables

In [35]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;



Column1          Col1 Col2 Col3     
-------          ---- ---- ----     
FD-PF33VX6J         1 Log  Shipping 
FD-PF33VX6J         2 Is   Resilient
FD-PF33VX6J\TONY    1 Log  Shipping 
FD-PF33VX6J\TONY    2 Is   Resilient




## Stop Primary SQL Server Service and Recover Secondary Database
This step simulates a failure of the primary SQL instance and recovers the database on the secondary.

In [36]:
Stop-Service -Name MSSQLSERVER -Force;
Stop-DbaProcess -SqlInstance 'localhost\tony' -Database LSTestDR;

Invoke-DbaDbLogShipRecovery -SqlInstance 'localhost\tony' -Force;



SqlInstance : localhost\tony
Spid        : 76
Login       : AzureAD\FrankGill
Host        : FD-PF33VX6J
Database    : LSTestDR
Program     : .Net SqlClient Data Provider
Status      : Killed



log shipping.
log shipping.
 shipping.
shipping.


database for log shipping.
log shipping.


ComputerName  : FD-PF33VX6J
InstanceName  : TONY
SqlInstance   : FD-PF33VX6J\TONY
Database      : LSTestDR
RecoverResult : Success
Comment       : 





## Select from New Primary Table

In [37]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@

$primary = "localhost\tony"
$primarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;


Column1          Col1 Col2 Col3     
-------          ---- ---- ----     
FD-PF33VX6J\TONY    1 Log  Shipping 
FD-PF33VX6J\TONY    2 Is   Resilient




# Restart Primary SQL Service

In [38]:
Start-Service -Name MSSQLSERVER;
Start-Service -Name SQLSERVERAGENT;